## MNIST handwritten digits

http://yann.lecun.com/exdb/mnist/

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import os, sys
import itertools, functools
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import scale, LabelBinarizer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

np.random.seed(18937)

## Keras
Keras is an abstraction of the tensorflow api to facilitate more easily constructed models. And actually, it is a general python library for model construction that support tensorflow and some other underlying libraries

In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam

Using TensorFlow backend.


https://keras.io/

We'll construct a convolutional neural network (http://deeplearning.net/tutorial/lenet.html) that has the following structure:
* Convolution with 5x5 pixel kernels, 32 of them, and using the Recitified Linear Unit (https://en.wikipedia.org/wiki/Rectifier_(neural_networks))
* Max pooling with 2x2 kernel. Find the strongest response in each 2x2 neuron area of a generated feature map (from the convolution)
* Convolve with 64 5x5 kernels, then max pooling again
* Stretch all the feature maps out into a vector
* A feed forward, fully connected layer
* 10 class activation using SoftMax, a logit layer, with all neurons normalized to sum to 1.0 (https://en.wikipedia.org/wiki/Softmax_function)

In [7]:
layers = [
    Input(shape=(784, )),
    Reshape((28, 28, 1)),
    
    # convolutional network
    Conv2D(32, (5, 5), activation = "relu", padding = "SAME"),
    MaxPooling2D((2, 2), strides = (2, 2), padding = "SAME"),
    Conv2D(64, (5, 5), activation = "relu", padding = "SAME"), 
    MaxPooling2D((2, 2), strides = (2, 2), padding = "SAME"),
    
    # fully connected network
    Reshape((7*7*64,)),
    Dense(1024, activation = "relu"),
    Dropout(0.5),
    Dense(10, activation = "softmax"),
]

y_pred = functools.reduce(lambda f1, f2: f2(f1), layers)

model = Model(inputs = [layers[0]], outputs = [y_pred])
model.compile(optimizer = Adam(), loss = "categorical_crossentropy", metrics = ["categorical_accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 3136)              0         
__________

## Load dataset

In [8]:
from tensorflow.examples.tutorials import mnist
dataset = mnist.input_data.read_data_sets("datasets/MNIST_data", one_hot = True)

Extracting datasets/MNIST_data\train-images-idx3-ubyte.gz
Extracting datasets/MNIST_data\train-labels-idx1-ubyte.gz
Extracting datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting datasets/MNIST_data\t10k-labels-idx1-ubyte.gz


## Train the model

In [9]:
model.fit(x = [dataset.train.images], y = [dataset.train.labels], batch_size = 50, epochs = 10, 
         validation_data = (dataset.validation.images, dataset.validation.labels), shuffle = True, verbose = 1)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 316s - loss: 0.1282 - categorical_accuracy: 0.9612 - val_loss: 0.0409 - val_categorical_accuracy: 0.9860
Epoch 2/10
55000/55000 [==============================] - 294s - loss: 0.0409 - categorical_accuracy: 0.9871 - val_loss: 0.0342 - val_categorical_accuracy: 0.9902
Epoch 3/10
55000/55000 [==============================] - 347s - loss: 0.0298 - categorical_accuracy: 0.9908 - val_loss: 0.0326 - val_categorical_accuracy: 0.9898
Epoch 4/10
55000/55000 [==============================] - 335s - loss: 0.0231 - categorical_accuracy: 0.9929 - val_loss: 0.0458 - val_categorical_accuracy: 0.9860
Epoch 5/10
55000/55000 [==============================] - 356s - loss: 0.0196 - categorical_accuracy: 0.9939 - val_loss: 0.0388 - val_categorical_accuracy: 0.9912
Epoch 6/10
55000/55000 [==============================] - 321s - loss: 0.0155 - categorical_accuracy: 0.9953 - val_loss: 0.0338 - val_ca

## Evaluate the model

In [11]:
print("Evaluation: \n")
print("loss: %.4f accuracy: %.4f" %
         tuple(model.evaluate(x = [dataset.test.images], y = [dataset.test.labels], batch_size = 50, verbose = 2)))

Evaluation: 

loss: 0.0314 accuracy: 0.9918
